# Comparing Different Models

In [11]:
import numpy as np
import torch
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

## Loading Data

First we need to load the testing datasets for each model

In [12]:
# load datasets from the folder
data_dir = '../data/'
image_sizes = ['4-4', '8-8', '16-16', '25-25', '32-32']
dirs = [data_dir + size + '/temp_class/test/' for size in image_sizes]

# define the transforms
dset_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()])

# load the datasets
dsets = {y: datasets.ImageFolder(x, dset_transform) for x in dirs for y in image_sizes}


## Load the models

In [13]:
model_dir = 'models/'
model_names = [size + '-NN.pth' for size in image_sizes]
model_paths = [model_dir + name for name in model_names]

# load the models
models = {y: torch.load(x) for x in model_paths for y in image_sizes}

### Visualizing the Performance of Different Models

First we define some helper functions to format the data

In [14]:
def get_predictions(model, data):
    temp_pred_below = []
    temp_pred_above = []
    model.eval()
    for idx in range(len(data)):
        image, label = data[idx]
        predicted_output = model(image.unsqueeze(0))

        # Need to apply softmax to get probabilities
        prob_output = torch.nn.functional.softmax(predicted_output).detach().numpy()
        predicted_label = torch.argmax(predicted_output).item()
        # print(f"predicted label: {predicted_label}, actual label: {label}")

        #get temp from label
        temp = data.classes[label]
        temp_pred_below.append([float(temp), prob_output[0][1]])
        temp_pred_above.append([float(temp), prob_output[0][0]])
    
    return temp_pred_below, temp_pred_above

In [15]:
def get_avg_err(temp_pred):
    temp_pred_avg = {}

    temp_pred_std = {}

    for temp, pred in temp_pred:
        if temp in temp_pred_avg:
            temp_pred_avg[temp].append(pred)
        else:
            temp_pred_avg[temp] = [pred]

        if temp in temp_pred_std:
            temp_pred_std[temp].append(pred)
        else:
            temp_pred_std[temp] = [pred]


    temp_pred_avg = {temp: np.mean(pred) for temp, pred in temp_pred_avg.items()}

    temp_pred_err = {temp: np.std(pred) for temp, pred in temp_pred_std.items()}

    return temp_pred_avg, temp_pred_err

Now we can plot the data

In [16]:
# get predictions for each model
temp_pred = {y: get_predictions(models[y], dsets[y]) for y in image_sizes}

#get the average and error for each model and plot it
for size in image_sizes:
    temp_pred_below, temp_pred_above = temp_pred[size]
    temp_pred_below_avg, temp_pred_below_err = get_avg_err(temp_pred_below)
    temp_pred_above_avg, temp_pred_above_err = get_avg_err(temp_pred_above)

    plt.errorbar(temp_pred_below_avg.keys(), temp_pred_below_avg.values(), yerr=temp_pred_below_err.values(), label='Below')
    plt.errorbar(temp_pred_above_avg.keys(), temp_pred_above_avg.values(), yerr=temp_pred_above_err.values(), label='Above')
    plt.title(f'Predictions for {size}x{size} images')
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.legend()
    plt.show()

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'